In [1]:
import re
from docx import Document
import os

def remove_inserts_and_format_headings(file_path, save_path):
    # Открываем документ
    doc = Document(file_path)
    
    # Регулярные выражения для поиска вставок, которые нужно удалить
    patterns = [
        r'\d+o',  # например, '4o'
        r'Вы сказали:',  # например, 'Вы сказали:'
        r'Chapter_\d+\.docx',  # например, 'Chapter_2.docx', 'Chapter_3.docx'
        r'Документ',  # например, 'Документ'
        r'ChatGPT'  # например, 'ChatGPT'
    ]

    # Регулярное выражение для заголовков глав: Глава + номер главы
    chapter_heading_pattern = re.compile(r'^Глава\s+\d+')

    # Функция для проверки, нужно ли удалять абзац
    def should_delete(paragraph):
        for pattern in patterns:
            if re.match(pattern, paragraph.text.strip()):
                return True
        return False

    # Создаем новый документ для сохранения изменений
    new_doc = Document()

    # Проходим по абзацам и удаляем те, которые соответствуют шаблонам
    for para in doc.paragraphs:
        if not should_delete(para):
            # Если абзац является заголовком главы, применяем стиль "Заголовок 3"
            if chapter_heading_pattern.match(para.text.strip()):
                new_para = new_doc.add_paragraph(para.text, style='Heading 3')
            else:
                # Добавляем абзац в новый документ, сохраняя форматирование
                new_para = new_doc.add_paragraph()
                for run in para.runs:
                    new_run = new_para.add_run(run.text)
                    # Сохраняем форматирование
                    new_run.bold = run.bold
                    new_run.italic = run.italic
                    new_run.underline = run.underline
                    new_run.font.name = run.font.name
                    new_run.font.size = run.font.size

    # Сохраняем новый документ
    new_doc.save(save_path)

    # Уведомление о сохранении файла
    abs_path = os.path.abspath(save_path)
    print(f"Файл успешно сохранен по адресу: {abs_path}")

# Пример использования
file_path = 'hateful_surrogate/summary.docx'  # Исходный файл
save_path = 'hateful_surrogate/summary_clean.docx'  # Файл для сохранения без вставок

remove_inserts_and_format_headings(file_path, save_path)


Файл успешно сохранен по адресу: C:\Users\ollga\Books_analysis\hateful_surrogate\summary_clean.docx
